In [5]:
import os
import numpy as np
from numpy import eye, asarray, dot, sum, diag
from numpy.linalg import svd

# input raw data, center, apply weights
# (raw_data(i) - raw_data(mean)) * sqrt(weight)
# http://stats.stackexchange.com/questions/113485/weighted-principal-components-analysis

unweighted_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/sample_case_work/GoPro/gopro_raw_data_v1.csv'
weights_filename = '/Users/pniessen/Rosetta_Desktop/Segmentation_2-point-0/segmentr/web/gopro_weights.csv'

X_names = list(np.genfromtxt(unweighted_filename, delimiter=',', names=True).dtype.names)
X_unweighted = np.asmatrix(np.genfromtxt(unweighted_filename, delimiter=',', skip_header=1))
X_weights = np.asmatrix(np.genfromtxt(weights_filename, delimiter=',', skip_header=1))

X_unweighted_mean = (np.mean(X_unweighted, axis = 0))
X_unweighted_centered = X_unweighted - X_unweighted_mean
X_weighted = np.multiply(X_unweighted_centered.T,np.sqrt(X_weights))
X = X_weighted.T

#following http://sebastianraschka.com/Articles/2014_pca_step_by_step.html
# correlation matrix
X_cor_mat = np.corrcoef(X.T)

# get eigenvectors and eigenvalues for the from the correlation matrix
eig_val_cor, eig_vec_cor = np.linalg.eig(X_cor_mat)

# initialize empty loading_factor_matrix of (0 x length of eigenvector)
eig_vec_cors_length = eig_vec_cor[:,1].shape[0]
loading_factors_matrix_3 = np.asmatrix(np.zeros((0,eig_vec_cors_length))) # matrix shape must be tupple

# filter for eigvec > 1.0, generate loading factor, add to loading_factors_matrix
for i in range(len(eig_val_cor)):
    one_eigenvector = eig_vec_cor[:,i].reshape(1,eig_vec_cors_length).T # unit vector, direction
    one_eigenvalue = eig_val_cor[i] # scale
    
    if one_eigenvalue >=1:
        # http://stats.stackexchange.com/questions/143905/loadings-vs-eigenvectors-in-pca-when-to-use-one-or-another
        loading_factor = one_eigenvector * np.sqrt(one_eigenvalue)
        loading_factors_matrix_3 = np.r_[loading_factors_matrix_3,loading_factor.T]

print loading_factors_matrix_3.shape
print loading_factors_matrix_3.T[:,1].sum()
print abs(loading_factors_matrix_3.T).sum()
        
# now rotate loading_factor_matrix - this helps to filter out 'midrange' factors
# http://stackoverflow.com/questions/17628589/perform-varimax-rotation-in-python-using-numpy
def varimax(Phi, gamma = 1.0, q = 100, tol = 1e-10):
    # gamma = 1.0: varimax
    # gamma = 0.0: quartimax  (https://github.com/rossfadely/consomme/blob/master/consomme/rotate_factor.py)

    p,k = Phi.shape;print 'p', p; print 'k', k
    R = eye(k) # identity matrix
    d=0
    for i in xrange(q):
        d_old = d
        print d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        #print i
        print "i: ",i
        print d / d_old
        
        if d_old!=0 and d/d_old < 1 + tol: 
            print "convergence in: ", i, "iterations"
            break
    return dot(Phi, R)

loading_factors_matrix_rotated_3 = varimax(loading_factors_matrix_3.T)
print loading_factors_matrix_rotated_3.shape
print abs(loading_factors_matrix_rotated_3[:,0]).sum()
print abs(loading_factors_matrix_rotated_3).sum()

# tested vs SPSS output using gopro data
# for explanation of SPSS output
# see http://www.ats.ucla.edu/stat/spss/output/principal_components.htm
# ----------------
# SPSS loading factor matrix files:
# spss_unrotated: (148, 32) 468.803663037
# spss_rotated: (148, 32) sum: 381.386018769
# ----------------
# Homebrew loading factor matrix files:
# homebrew_unrotated: (148, 32) sum: 468.845148151
# homebrew_rotated: (148, 32) sum: 380.83076567
# -----pairwise comparison-------------
# unrotated loading_factors_matrix are similar using np.isclose @ atol = 1e-04
# rotated loading_factors_matrix are similar using np.isclose @ atol = 1e-04


(32, 148)
5.93393841669
468.845148151
0
i:  0
inf
5.16253926836
i:  1
1.83818417567
9.48969798936
i:  2
1.45796117773
13.8356112569
i:  3
1.20886213491
16.7253465617
i:  4
1.12278226546
18.7789225032
i:  5
1.10199685897
20.6943136134
i:  6
1.04051044391
21.5326494444
i:  7
1.01803428823
21.9209754509
i:  8
1.01117527336
22.1659483439
i:  9
1.00605705367
22.3002086826
i:  10
1.00339624107
22.3759455671
i:  11
1.00202112019
22.4211700425
i:  12
1.00137261344
22.4519456419
i:  13
1.00093473292
22.4729322146
i:  14
1.00055755999
22.4854622225
i:  15
1.00030786595
22.4923847308
i:  16
1.00017488935
22.4963184093
i:  17
1.00010514311
22.4986837422
i:  18
1.0000657859
22.5001638383
i:  19
1.00004214219
22.5011120445
i:  20
1.00002738796
22.501728304
i:  21
1.00001796625
22.5021325755
i:  22
1.00001186012
22.5023994536
i:  23
1.00000786385
22.5025764091
i:  24
1.00000523107
22.5026941217
i:  25
1.00000348863
22.5027726253
i:  26
1.00000233161
22.5028250929
i:  27
1.00000156134
22.5028602274
i:

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:65: RuntimeWarning: divide by zero encountered in double_scalars
